### velocity, rpm, plate_x, plate_z, L/R, movement, pitch_type -> 'woba_value'

In [65]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from pybaseball import statcast #https://github.com/jldbc/pybaseball

### Questions:
1. How does mlb get the colored batter box?

### Inspiration:
1. https://github.com/jwilsonds/swing_probability_model/blob/master/votto%20swing%20probability.ipynb 



https://www.kaggle.com/code/nickwan/attack-zones-visualizing-the-strike-zone

https://baseballsavant.mlb.com/csv-docs

In [66]:
from pybaseball import  playerid_lookup
from pybaseball import  statcast_pitcher
from pybaseball import  statcast_batter

In [67]:
#lookup for a player using their name. useful to find their id
#id = key_mlbam
playerid_lookup('soto', 'juan')

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,soto,juan,665742,sotoj001,sotoju01,20123,2018.0,2024.0


In [68]:
#juan soto's id is 665742
#find juan soto's stats from '2023-04-21' to '2024-04-21'
soto_stats = statcast_batter('2023-04-21', '2024-04-21', 665742)
soto_stats

Gathering Player Data


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,CU,2024-04-21,76.7,-0.78,6.51,"Soto, Juan",665742,650644,field_out,hit_into_play,...,1,1,1,1,1,Infield shade,Standard,43.0,-0.029,-0.397
1,SI,2024-04-21,91.1,-1.13,6.24,"Soto, Juan",665742,650644,NaN,ball,...,1,1,1,1,1,Infield shade,Standard,209.0,0.000,0.123
2,SI,2024-04-21,92.2,-1.14,6.23,"Soto, Juan",665742,650644,NaN,called_strike,...,1,1,1,1,1,Infield shade,Standard,210.0,0.000,-0.064
3,FF,2024-04-21,91.2,-0.93,6.45,"Soto, Juan",665742,650644,NaN,ball,...,1,1,1,1,1,Infield shade,Standard,206.0,0.000,0.072
4,FS,2024-04-21,84.7,-1.18,6.23,"Soto, Juan",665742,650644,NaN,ball,...,1,1,1,1,1,Infield shade,Standard,232.0,0.000,0.038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,FC,2023-04-21,88.5,-2.93,5.83,"Soto, Juan",665742,668678,NaN,called_strike,...,0,0,0,0,0,Infield shade,Standard,185.0,0.000,-0.038
3119,KC,2023-04-21,84.7,-2.93,5.86,"Soto, Juan",665742,668678,field_out,hit_into_play,...,0,0,0,0,0,Infield shade,Standard,44.0,0.015,-0.196
3120,CH,2023-04-21,85.0,-2.97,5.81,"Soto, Juan",665742,668678,NaN,ball,...,0,0,0,0,0,Infield shade,Standard,228.0,0.000,0.034
3121,CH,2023-04-21,87.7,-3.09,5.80,"Soto, Juan",665742,668678,NaN,called_strike,...,0,0,0,0,0,Infield shade,Standard,221.0,0.000,-0.031


In [69]:
#remove rows where woba_denom is 0.
#because it includes sac_bunt and caught_stealing_2b
soto_stats = soto_stats[soto_stats['woba_denom']!=0]
soto_stats

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,CU,2024-04-21,76.7,-0.78,6.51,"Soto, Juan",665742,650644,field_out,hit_into_play,...,1,1,1,1,1,Infield shade,Standard,43.0,-0.029,-0.397
1,SI,2024-04-21,91.1,-1.13,6.24,"Soto, Juan",665742,650644,NaN,ball,...,1,1,1,1,1,Infield shade,Standard,209.0,0.000,0.123
2,SI,2024-04-21,92.2,-1.14,6.23,"Soto, Juan",665742,650644,NaN,called_strike,...,1,1,1,1,1,Infield shade,Standard,210.0,0.000,-0.064
3,FF,2024-04-21,91.2,-0.93,6.45,"Soto, Juan",665742,650644,NaN,ball,...,1,1,1,1,1,Infield shade,Standard,206.0,0.000,0.072
4,FS,2024-04-21,84.7,-1.18,6.23,"Soto, Juan",665742,650644,NaN,ball,...,1,1,1,1,1,Infield shade,Standard,232.0,0.000,0.038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,FC,2023-04-21,88.5,-2.93,5.83,"Soto, Juan",665742,668678,NaN,called_strike,...,0,0,0,0,0,Infield shade,Standard,185.0,0.000,-0.038
3119,KC,2023-04-21,84.7,-2.93,5.86,"Soto, Juan",665742,668678,field_out,hit_into_play,...,0,0,0,0,0,Infield shade,Standard,44.0,0.015,-0.196
3120,CH,2023-04-21,85.0,-2.97,5.81,"Soto, Juan",665742,668678,NaN,ball,...,0,0,0,0,0,Infield shade,Standard,228.0,0.000,0.034
3121,CH,2023-04-21,87.7,-3.09,5.80,"Soto, Juan",665742,668678,NaN,called_strike,...,0,0,0,0,0,Infield shade,Standard,221.0,0.000,-0.031


#### Values in use currently
woba_value(weighted on base average):

#not sure what the values should be here
- field out/strikeout: 0.0
- walk: 0.70
- field error: 0.9 (prob remove)
- single: 0.90
- double: 1.25
- triple: 1.60
- home_run: 2.0


- ball: 0.175
- strike: 0 (is this bad? strike same value as strikeout)

In [70]:
#assigns woba_value to strikes and balls
#strike = 0 and ball = 0.175
#ignores foul balls
rows_to_update_S = (soto_stats['description'] == 'called_strike') | \
(soto_stats['description'] == 'swinging_strike') | \
(soto_stats['description'] == 'swinging_strike_blocked')
soto_stats.loc[rows_to_update_S, 'woba_value'] = 0
rows_to_update_B = soto_stats['type']=='B'
soto_stats.loc[rows_to_update_B, 'woba_value'] = 0.175

In [71]:
#remove all rows where woba_value is nan.
soto_stats = soto_stats[~soto_stats['woba_value'].isna()]

### Prepare data for model
i followed this xgboost tutorial: https://www.datacamp.com/tutorial/xgboost-in-python

#### Inputs
    -Velocity
    -Spin Rate
    -Pitcher Left or Right Handed
    -Pitch Type
    -Pitch movement
    -Pitch location

In [72]:
X = soto_stats[['release_speed', 'release_spin_rate', 'p_throws','pitch_type', \
'pfx_x', 'pfx_z', 'plate_x', 'plate_z']]

#allows xgboost to take care of categorical variables.
X['pitch_type'] = X['pitch_type'].astype('category')
X['p_throws'] = X['p_throws'].astype('category')
X

/var/folders/21/3pfj48b93_dfj5qvpfdk7v0h0000gn/T/ipykernel_78287/135672852.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['pitch_type'] = X['pitch_type'].astype('category')
/var/folders/21/3pfj48b93_dfj5qvpfdk7v0h0000gn/T/ipykernel_78287/135672852.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['p_throws'] = X['p_throws'].astype('category')


,release_speed,release_spin_rate,p_throws,pitch_type,pfx_x,pfx_z,plate_x,plate_z
0,76.7,2964.0,R,CU,1.07,-1.35,-0.17,2.25
1,91.1,2176.0,R,SI,-1.06,1.31,-1.88,2.23
2,92.2,2271.0,R,SI,-1.10,1.18,0.29,3.21
3,91.2,2336.0,R,FF,-0.39,1.55,0.66,3.95
4,84.7,2015.0,R,FS,-1.48,0.72,-0.59,1.35
...,...,...,...,...,...,...,...,...
3118,88.5,2431.0,R,FC,0.26,0.66,0.15,2.11
3119,84.7,2611.0,R,KC,0.27,-0.89,-0.36,2.06
3120,85.0,1565.0,R,CH,-1.00,0.18,-0.04,1.47
3121,87.7,1680.0,R,CH,-1.12,0.24,-0.51,1.80


In [73]:
soto_stats[soto_stats['woba_value'].isna()]

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp


In [74]:
y = soto_stats['woba_value']

In [75]:
#because most pitches are balls and strikes so it brings the mean down
np.mean(soto_stats['woba_value'])

0.1740623839584107

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40)

### Create the model

how gbm works for regression: https://www.youtube.com/watch?v=3CC4N4z3GJc


In [77]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

In [78]:
#moving data to XGBoost's dataset class to improve memory and speed
dtrain = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [79]:
params = {"objective": "reg:squarederror"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)
model

In [80]:
predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
rmse

0.34746795845006473

In [81]:
mae = mean_absolute_error(y_test, predictions)
mae

0.1974444587355879

In [ ]:
#combining stand and p_throws columns.
#now unneccessary



# soto_stats[['stand', 'p_throws']]
# #func createLeftRight combines stand and p_throws
# #if stand = 'L' and p_throws = 'R', then createLeftRight will return 'LR'
# def createLeftRight(stand, p_throws):
#     if stand == 'L':
#         if p_throws == 'R':
#             return 'LR'
#         elif p_throws == 'L':
#             return 'LL'
#     elif stand == 'R':
#         if p_throws == 'R':
#             return 'RR'
#         elif p_throws == 'L':
#             return 'RL'
        
# soto_stats['L/R'] = soto_stats[['stand', 'p_throws']]\
# .apply(lambda x: createLeftRight(x['stand'], x['p_throws']), axis = 1)
# soto_stats[['stand', 'p_throws', 'L/R']]